In [184]:
import os
import re
import itertools
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit import RDConfig
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
from rdkit.Chem import FragmentCatalog
from rdkit.Chem import DataStructs
import heapq
from rdkit.Chem import Descriptors
from rdkit.Chem import BRICS
from rdkit.ML.Descriptors import MoleculeDescriptors
from itertools import chain
import math

In [ ]:
directory = 'G:/My Drive/NCSU/DiamondHacks/'


fName= 'C:/RDKit_2017_03_2/Data/FunctionalGroups.txt'
fparams = FragmentCatalog.FragCatParams(1,6,fName)
fcat=FragmentCatalog.FragCatalog(fparams)

##        macrocycle_file = 'macrocycles_IDs.csv'
##        suppl = [i.split(',')[0] for i in open(self.directory+name,'r').read().splitlines()][1:]       # read all the macrocycle smiles from file
##        ms = [Chem.MolFromSmiles(i) for i in suppl]     # mols of macrocycles

zinc_file = 'smiles_database.csv'
zinc_suppl = [i.split(',')[1] for i in open(directory + zinc_file,'r').read().splitlines()][1:]
zinc_ms = [Chem.MolFromSmiles(i) for i in zinc_suppl]

fcgen=FragmentCatalog.FragCatGenerator()  # fragment generator

#for m in ms:
#    fcgen.AddFragsFromMol(m,self.fcat)

for m in zinc_ms:
    fcgen.AddFragsFromMol(m,fcat)
    

In [8]:
fcat.GetNumEntries()

143377

In [11]:
frag_database = [fcat.GetEntryDescription(i) for i in range(fcat.GetNumEntries())]

In [12]:
frg_manager = open(directory + 'ZINC_fragments_10k','w')
to_write = '\n'.join(frag_database)
frg_manager.write(to_write)

2414060

In [75]:
frg_manager.close()

In [14]:
frag_database

['C<-SCH3>c',
 'cc',
 'cC<=O>',
 'C<=O>N',
 'cN',
 'cn',
 'ccC<-SCH3>',
 'ccc',
 'ccC<=O>',
 'cC<=O>N',
 'C<=O>Nc',
 'ncN',
 'ccN',
 'ccn',
 'cnc',
 'cccC<-SCH3>',
 'cc(c)C<-SCH3>',
 'cccc',
 'cccC<=O>',
 'cc(c)C<=O>',
 'ccC<=O>N',
 'cC<=O>Nc',
 'C<=O>Ncn',
 'ccNC<=O>',
 'cncN',
 'cc(n)N',
 'cccN',
 'cccn',
 'ccnc',
 'ccccC<-SCH3>',
 'ccc(c)C<-SCH3>',
 'ccccc',
 'ccccC<=O>',
 'ccc(c)C<=O>',
 'cccC<=O>N',
 'cc(c)C<=O>N',
 'ccC<=O>Nc',
 'cC<=O>Ncn',
 'ccNC<=O>c',
 'cncNC<=O>',
 'cc(n)NC<=O>',
 'cccNC<=O>',
 'ccncN',
 'cnc(c)N',
 'ccc(n)N',
 'ccccN',
 'ccccn',
 'cccnc',
 'ccncc',
 'C<-SCH3>ccccC<=O>',
 'cccccC<-SCH3>',
 'cccc(c)C<-SCH3>',
 'ccc(C<-SCH3>)cc',
 'cccccc',
 'cccc(c)C<=O>',
 'ccccC<=O>N',
 'cccccC<=O>',
 'ccc(C<=O>)cc',
 'ccc(c)C<=O>N',
 'cccC<=O>Nc',
 'cc(c)C<=O>Nc',
 'ccC<=O>Ncn',
 'ccC<=O>Ncc',
 'cncNC<=O>c',
 'cc(n)NC<=O>c',
 'cccNC<=O>c',
 'ccncNC<=O>',
 'cnc(c)NC<=O>',
 'ccc(n)NC<=O>',
 'ccccNC<=O>',
 'cccncN',
 'ccnc(c)N',
 'ccc(N)nc',
 'cccc(n)N',
 'cccccN',
 'cccccn',

In [18]:
def fragment_simple(mol, atom1, atom2):
    rwmol = Chem.RWMol(mol)
    rwmol.RemoveBond(atom1, atom2)
    wildcard1 = rwmol.AddAtom(Chem.Atom(0))
    wildcard2 = rwmol.AddAtom(Chem.Atom(0))
    rwmol.AddBond(atom1, wildcard1, Chem.BondType.SINGLE) 
    rwmol.AddBond(atom2, wildcard2, Chem.BondType.SINGLE) 
    return rwmol.GetMol()

vanillin = Chem.MolFromSmiles("c1(C=O)cc(OC)c(O)cc1")
fragmented_vanillin = fragment_simple(vanillin,4,5)

In [19]:
Chem.MolToSmiles(fragmented_vanillin)

'[*]OC.[*]c1cc(C=O)ccc1O'

In [22]:
pre_synthetic_frag_database = [BRICS.BRICSDecompose(i) for i in zinc_ms]

In [23]:
pre_synthetic_frag_database

[{'[1*]C([6*])=O',
  '[11*]SC',
  '[14*]c1ccccn1',
  '[16*]c1ccc([16*])cc1',
  '[4*]C[8*]',
  '[5*]N[5*]'},
 {'[1*]C([6*])=O',
  '[14*]c1nc2cc([16*])ccc2[nH]1',
  '[16*]c1cccc(F)c1',
  '[3*]OC',
  '[4*]C[8*]',
  '[5*]N[5*]'},
 {'[1*]C([6*])=O',
  '[14*]c1cccc(=O)[nH]1',
  '[16*]c1ccccc1',
  '[5*]N1C[C@H]([15*])C(C)(C)C1'},
 {'[14*]c1cc([14*])on1',
  '[16*]c1ccncc1',
  '[16*]c1ccsc1',
  '[4*]C[8*]',
  '[5*]N([5*])C'},
 {'[1*]C([1*])=O', '[16*]c1ccc(F)cc1', '[16*]c1cccc(C#N)c1', '[5*]N[5*]'},
 {'[1*]C([6*])=O', '[14*]c1oc2c(F)cccc2c1C', '[5*]N1CCCSCC1'},
 {'[14*]c1ccc([14*])o1',
  '[14*]c1ccccn1',
  '[4*]C[8*]',
  '[5*]N1CC([15*])C1',
  '[8*]CCC'},
 {'[10*]N1C[C@]([15*])(C)CCC1=O',
  '[7*]C(C)C',
  '[7*]CCC[8*]',
  '[8*]CC[8*]',
  '[9*]n1ccnc1'},
 {'[1*]C([1*])=O',
  '[16*]c1c(C)noc1C',
  '[16*]c1ccccc1',
  '[4*]CC',
  '[4*]C[8*]',
  '[5*]N([5*])[5*]',
  '[5*]N[5*]'},
 {'[1*]C([6*])=O', '[14*]c1oc2ccccc2c1C', '[5*]N1CCCSCC1'},
 {'[10*]N1C[C@@]([15*])(C)CCC1=O',
  '[7*]C(C)C',
  '[7*]CCC[

In [24]:

synthetic_frag_database = list(set(chain.from_iterable(pre_synthetic_frag_database)))

In [25]:
synthetic_frag_database

['[5*]N1CCC[C@]([15*])(C)CC1',
 '[16*]c1ccc2[nH]cnc2c1',
 '[13*][C@H]1COc2ccccc2O1',
 '[16*]c1cc([16*])c([N+](=O)[O-])cc1[16*]',
 '[15*][C@H]1[C@@H]2CCO[C@@H]2[C@@H]1[15*]',
 '[4*]C[C@](C)(O)C([8*])[8*]',
 '[4*]C1([15*])CCC1',
 '[16*]c1c([N+](=O)[O-])c(=O)oc2ccccc12',
 '[16*]c1cc(Cl)c(N)c(Cl)c1',
 '[16*]c1ccc(F)c(Br)c1',
 '[8*]C[C@@H](CC)C(=O)CC',
 '[14*]c1sc2c(c1C#N)CCC2',
 '[5*][N@H+]1CCC[C@@H]([15*])C1',
 '[9*]n1c([14*])nc2ccccc21',
 '[5*]N1C[C@H]2CCCC[C@@]21[13*]',
 '[16*]c1ccc2c(c1)OCCC2=O',
 '[14*]c1ncccc1C=O',
 '[15*][C@@H]1C[C@H]1C',
 'CCCC[S@@](=O)c1cccc(Br)c1',
 '[9*]n1nc([14*])oc1=S',
 '[16*]c1c(C)nsc1N',
 '[15*][C@H]1CC[C@@H](C)C[C@@H]1[15*]',
 '[9*]n1nc([14*])n2c3ccccc3nc12',
 '[6*]C(=O)n1nc([14*])c2c1CC(C)(C)CC2=O',
 '[13*][C@@H]1Oc2ccccc2O[C@@H]1C',
 '[16*]c1ccc(Cl)c(Cl)c1',
 '[16*]c1ccc([N+](=O)[O-])cc1C',
 '[14*]c1cc(C)nc2c(Cl)cnn12',
 '[16*]c1c(Cl)nc(C)c(Cl)c1C',
 '[5*]N1CCc2ccccc2C1',
 '[15*][C@@H]1CC1(Cl)Cl',
 '[15*][C@@H]1CCC[C@H](C)C1',
 '[14*]c1c([16*])c(C)nc2nc(

In [70]:
new_synthetic_frag_database = list(set(synthetic_frag_database))
len(new_synthetic_frag_database), len(synthetic_frag_database)

(3110, 3110)

In [ ]:
all_replaceable_fragments

In [27]:
frg_manager = open(directory + 'ZINC_fragments_10k_synthetic','w')
to_write = '\n'.join(synthetic_frag_database)
frg_manager.write(to_write)

77893

In [28]:
test_mol = Chem.MolFromSmiles(synthetic_frag_database[0])

In [33]:
def prep_target(chemical):
    target_fragments = BRICS.BRICSDecompose(Chem.MolFromSmiles(chemical))

ERY = 'CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(OC)[C@@H](O)[C@H](C)O2)[C@H](C)[C@@H](O[C@@H]2O[C@H](C)C[C@@H]([C@H]2O)N(C)C)[C@](C)(O)C[C@@H](C)C(=O)[C@H](C)[C@@H](O)[C@]1(C)O'
ERY_fragments = list(BRICS.BRICSDecompose(Chem.MolFromSmiles(ERY)))

quinine = 'COC1=CC2=C(C=CN=C2C=C1)[C@@H](O)[C@@H]1C[C@@H]2CCN1C[C@@H]2C=C'
quinine_fragments =  list(BRICS.BRICSDecompose(Chem.MolFromSmiles(quinine)))   #anti-malarial drug

In [34]:
quinine_fragments

['[13*][C@@H]1C[C@@H]2CCN1C[C@@H]2C=C',
 '[8*]C([8*])O',
 '[3*]OC',
 '[16*]c1ccc2nccc([16*])c2c1']

In [39]:
def eudis(v1, v2):
    dist = [(a - b)**2 for a, b in zip(v1, v2)]
    dist = math.sqrt(sum(dist))
    return dist

In [76]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])

def get_similar_fragments(target_cpd,n=20):
    score_frg= []
    for i in synthetic_frag_database:
        cur_frag_ds = [0 if math.isnan(x) else x for x in calc.CalcDescriptors(Chem.MolFromSmiles(i))]
        dst = eudis(cur_frag_ds,target_cpd)
        score_frg.append((i,dst))
    sorted_tuples = sorted(score_frg, key=lambda x: x[-1])
    #lowest_scores = heapq.nsmallest(n, [k[1] for k in score_frg])
    chosen_fragments = [k[0] for k in sorted_tuples[:n]]
    #chosen_fragments = [list(score_dict.keys())[list(score_dict.values()).index(j)] for j in lowest_scores]
    return chosen_fragments

all_replaceable_fragments = []
for i in quinine_fragments:
    target_ds =  [0 if math.isnan(x) else x for x in calc.CalcDescriptors(Chem.MolFromSmiles(i))]
    all_replaceable_fragments.append(get_similar_fragments(target_ds,50))
    
print(all_replaceable_fragments)

[['[5*]N1CC[C@H]2OCCC[C@H]2C1', '[5*]N1CC[C@@H]2OCCC[C@H]2C1', '[15*][C@H]1[C@@H]2CCO[C@@H]2C12CCC2', '[15*][C@@H]1[C@@H]2CCO[C@@H]2C12CCC2', '[15*][C@H]1[C@H]2CCO[C@@H]2C12CCC2', '[5*]N1CC[C@@H]2CCCC[C@@H]2C1', '[5*]N1CC[C@@H]2CCCC[C@H]2C1', '[5*]N1CCO[C@H]2CCCC[C@@H]21', '[5*]N1CCO[C@@H]2CCCC[C@H]21', '[5*]N1CCC[C@H]2CCCC[C@@H]21', '[5*]N1CCC[C@]2(CCOC2)C1', '[5*]N1CCC[C@@]2(CCOC2)C1', '[5*][N@@H+]1[C@H]([13*])C[C@@H]2CCCC[C@H]21', '[5*][N@@H+]1[C@H]2CCCC[C@H]2C[C@H]1[13*]', '[5*][N@@H+]1[C@H]2CCCC[C@@H]2C[C@H]1[13*]', '[13*][C@H]1CO[C@]2([13*])CCC(=O)N12', '[15*]C12C[C@H]3C[C@@H](C1)C[C@@H](C2)C3', '[15*]C12C[C@H]3C[C@H](C1)C[C@@H](C2)C3', '[14*]c1cc2c(s1)CCCCC2', '[15*][C@@H]1CC[C@H]2CCCC[C@@H]2C1', '[15*][C@@H]1CC[C@H]2CCCC[C@H]2C1', '[5*]N1C[C@]2(C)C[C@H]1CC(C)(C)C2', '[5*]NC(=S)N1C[C@H](C)O[C@H](C)C1', '[5*]NC(=S)N1C[C@@H](C)O[C@H](C)C1', '[5*]NC(=S)N1CCCCCC1', '[10*]N1C(=O)N2CCCN2[C@H]1[13*]', '[15*][C@]12C[C@@H]3C[C@H](C1)C[C@@](O)(C3)C2', '[15*][C@@H]1CCc2ccccc2C1', '[15*][C@

In [77]:
len(all_replaceable_fragments)

4

In [112]:
header = ['fragment '+str(u+1) for u in range(len(quinine_fragments))]
original_fragments = dict(zip(header, quinine_fragments))

In [113]:
original_fragments

{'fragment 1': '[13*][C@@H]1C[C@@H]2CCN1C[C@@H]2C=C',
 'fragment 2': '[8*]C([8*])O',
 'fragment 3': '[3*]OC',
 'fragment 4': '[16*]c1ccc2nccc([16*])c2c1'}

In [116]:
#target_manager = open(directory + 'ChemX/quinine_fragments','w')
#header = ','.join(['fragment '+str(u+1) for u in range(len(quinine_fragments))])+'\n'
#target_manager.write(header)

#for i in range(len(all_replaceable_fragments)):
 #   to_write = ','.join(all_replaceable_fragments[i])+'\n'
#frg_manager.write(to_write)

header = ['fragment '+str(u+1) for u in range(len(quinine_fragments))]
top_row = dict(zip(header, quinine_fragments))

cv_frame = pd.DataFrame(np.column_stack(all_replaceable_fragments), 
                               columns=header)  

data = []
data.insert(0,top_row)

pd.concat([pd.DataFrame(data), cv_frame], ignore_index=True)

,fragment 1,fragment 2,fragment 3,fragment 4
0,[13*][C@@H]1C[C@@H]2CCN1C[C@@H]2C=C,[8*]C([8*])O,[3*]OC,[16*]c1ccc2nccc([16*])c2c1
1,[5*]N1CC[C@H]2OCCC[C@H]2C1,[8*]C([8*])O,[3*]OC,[14*]c1cc([16*])c2ccccc2n1
2,[5*]N1CC[C@@H]2OCCC[C@H]2C1,[8*]CO,[5*][NH+]([5*])C,[14*]c1nc2cc(C)ccn2c1[14*]
3,[15*][C@H]1[C@@H]2CCO[C@@H]2C12CCC2,[8*]C([8*])([8*])O,[5*]NC,[14*]c1nc2cc([16*])ccc2n1C
4,[15*][C@@H]1[C@@H]2CCO[C@@H]2C12CCC2,[8*]C[NH3+],[5*]N([5*])C,[16*]c1ccc2[nH]c(C)c([16*])c2c1
5,[15*][C@H]1[C@H]2CCO[C@@H]2C12CCC2,[3*]OC,[8*]CO,[14*]c1nc([14*])c2ccccc2n1
6,[5*]N1CC[C@@H]2CCCC[C@@H]2C1,[5*]NC,[8*]C([8*])O,[14*]c1oc2ccc([16*])cc2c1C
7,[5*]N1CC[C@@H]2CCCC[C@H]2C1,[5*][NH+]([5*])C,[8*]C[NH3+],[16*]c1ccc2oc(C)c([16*])c2c1
8,[5*]N1CCO[C@H]2CCCC[C@@H]21,[5*]N([5*])C,[8*]C([8*])([8*])O,[14*]c1nc2ccc(C)cn2c1[14*]
9,[5*]N1CCO[C@@H]2CCCC[C@H]21,[1*]C([1*])=O,[3*]OC[8*],[16*]c1cccc2ccc(C)nc12


In [118]:
cv_frame.to_csv('ChemX/data/quinine_fragments',index=False)

In [197]:
potential_cpd_templates = list(itertools.product(*all_replaceable_fragments))
print(len(potential_cpd_templates))

6250000


In [143]:
sample = []
for i in range(len(all_replaceable_fragments)):
    sample.append(all_replaceable_fragments[i][0])
    print(all_replaceable_fragments[i][0])

[5*]N1CC[C@H]2OCCC[C@H]2C1
[8*]C([8*])O
[3*]OC
[14*]c1cc([16*])c2ccccc2n1


In [166]:
test = '[13*][C@@H]1C[C@@H]2CCN1C[C@@H]2C=C'
digits = re.findall(r'\d+', test)
print(digits)
#print([int(s) for s in test.split() if s.isdigit()])

['13', '1', '2', '1', '2']


In [189]:
mini_frags = []
for each in quinine_fragments:
    numjoints = each.count('*')
    numbers = re.findall(r'\d+', each)
    possible_joints = ['['+str(m) +'*]' for m in numbers] 
    #print(possible_joints)
    
    for i in range(numjoints):
        for j in possible_joints:
            if j in each:    
                littleones = each.replace(j,'').replace('()',',').rstrip().split(',')
                #print(littleones)
                if littleones not in mini_frags:
                    mini_frags.append(littleones)     
mini_frags_flattened = list(itertools.chain(*mini_frags))

In [190]:
mini_frags_flattened

['[C@@H]1C[C@@H]2CCN1C[C@@H]2C=C', 'C', 'O', 'OC', 'c1ccc2nccc', 'c2c1']

In [191]:
fragms = [Chem.MolFromSmiles(x) for x in sorted(sample)]

In [192]:
ms = BRICS.BRICSBuild(fragms)

In [193]:
ms

<generator object BRICSBuild at 0x0000015452C047D8>

In [194]:
prods = [next(ms) for x in range(20)]

In [195]:
for prod in prods:
    prod.UpdatePropertyCache(strict=False)

In [196]:
percent = len(mini_frags_flattened)
counter = 0
for i in range(20):
    for j in range(len(mini_frags_flattened)): 
        sampler = Chem.MolToSmiles(prods[i],True)
        if mini_frags_flattened[j] in sampler:
            counter+=1
            if counter == percent:
                print(sampler)
    

COc1cc(N2CC[C@H]3OCCC[C@H]3C2)c2ccccc2n1


In [198]:
quinine_fragments

['[13*][C@@H]1C[C@@H]2CCN1C[C@@H]2C=C',
 '[8*]C([8*])O',
 '[3*]OC',
 '[16*]c1ccc2nccc([16*])c2c1']

In [215]:
def collect_mini_frags_from_each_template(current_template):
    mini_frags = []
    for each in current_template:
        numjoints = each.count('*')
        numbers = re.findall(r'\d+', each)
        possible_joints = ['['+str(m) +'*]' for m in numbers]
        #print(possible_joints)

        for i in range(numjoints):
            for j in possible_joints:
                if j in each:
                    littleones = each.replace(j,'').replace('()',',').rstrip().split(',')
                    #print(littleones)
                    if littleones not in mini_frags:
                        mini_frags.append(littleones)
    mini_frags_flattened = list(itertools.chain(*mini_frags))
    return mini_frags_flattened



In [223]:
f = open('ChemX/data/quinine_analogue','a+')
for current_template in potential_cpd_templates:
    fragms = [Chem.MolFromSmiles(x) for x in current_template]
    ms = BRICS.BRICSBuild(fragms)
    prods = [next(ms) for x in range(1)]
    mini_frags = collect_mini_frags_from_each_template(current_template)
    percent = len(mini_frags)
    counter = 0
    for i in range(1):
        for j in range(len(mini_frags)):
            sampler = Chem.MolToSmiles(prods[i],True)
            if mini_frags[j] in sampler:
                counter+=1
                if counter == percent:
                    print(sampler)
                    f.write(sampler+'\n')

StopIteration: 

In [ ]:
f = open('ChemX/data/quinine_analogue','a+')
for current_template in potential_cpd_templates:
    fragms = [Chem.MolFromSmiles(x) for x in current_template]
    ms = BRICS.BRICSBuild(fragms)
    prods = [next(ms) for x in range(1)]
    mini_frags = collect_mini_frags_from_each_template(current_template)    
    for i in range(1):
        sampler = Chem.MolToSmiles(prods[i],True)
        print(sampler)
        f.write(sampler+'\n')